In [125]:
def predict(x):
    from pyspark.sql.functions import substring,conv,col,min,max,udf,lit
   
    d = spark.read.parquet('/data/user/hive/warehouse/ian/tmp_80AD16')
    
    bottom = d.select(min('p')).collect()[0]['min(p)']
    top = d.select(max('p')).collect()[0]['max(p)'] 
    
    if x < bottom:
        return d.filter(d.p == bottom).select('id').collect()[0]['p']
    
    elif x > top:
        return d.filter(d.p == top).select('id').collect()[0]['p']
    
    else :
        p1 = test.filter(test.p == x)
        p1 = p1.replace(p1.collect()[0]['id'],None)

        from pyspark.sql import Window
        from pyspark.sql.functions import row_number
        window = Window.orderBy('p')

        p1 = p1.unionByName(d1).sort('p',ascending=False)

        d2 = p1.withColumn('index', row_number().over(window))

        index = d2.filter(d2.p == 7431407).collect()[0]['index']

        df = d2.where(col('index').between(index-5,index+5)).drop_duplicates().dropna()

        tmp = df.groupBy('id').count()

        return tmp.sort('count',ascending=False).collect()[0]['id'] 

In [ ]:
import pandas as pd
from pyspark.sql.functions import substring,conv,col

test = spark.read.parquet('/data/user/hive/warehouse/ian/d6')

result = []
a = test.select('p').toPandas()['p'].values.tolist()
for x in a:
    result.append(predict(x))

mac = test.select('mac').toPandas()['mac'].values.tolist()
pre = spark.createDataFrame(pd.DataFrame({'mac':mac,'predict_id':result}))

pre = pre.withColumnRenamed('mac','mac1')
dd = test.join(pre,test.mac == pre.mac1,how='left')

from pyspark.sql.functions import when
dd = dd.withColumn('equal',when(dd.predict_id == dd.id, 1).otherwise(0))

dd.groupBy('equal').count().show()